In [45]:
from sia.satellite import sentinel2
from sia.utils.helper import *
from sia.utils.raster import *
from sia.indices.indice import *
import os
import errno
from collections import defaultdict
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
s2 = sentinel2.Sentinel2()

# # s2_tile = '/home/ec2-user/SageMaker/sumit/crop-classification/unsupcc/satellite/satellite_tiles/s2_tile.shp'
# shape_file = 'data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp'
shape_file = '/Users/sumitmaan/work/dehaat/india_shapefile/districts/vaishali/file.shp'
pids = s2.get_product_ids('2021-11-01', '2021-12-31', 10, 5, 
                       shape_file=shape_file, bbox=None)

In [47]:
'/Users/sumitmaan/work/dehaat/crop-health-monitor/vaishali.img'

'/Users/sumitmaan/work/dehaat/crop-health-monitor/vaishali.img'

In [48]:
pids = {'2021-11-03': ['S2A_45RUJ_20211103_0_L2A'],
 '2021-11-08': ['S2B_45RUJ_20211108_0_L2A'],
 '2021-11-13': ['S2A_45RUJ_20211113_0_L2A'],
 '2021-11-18': ['S2B_45RUJ_20211118_0_L2A'],
 '2021-11-23': ['S2A_45RUJ_20211123_0_L2A'],
 '2021-11-28': ['S2B_45RUJ_20211128_0_L2A'],
 '2021-12-03': ['S2A_45RUJ_20211203_0_L2A'],
 '2021-12-08': ['S2B_45RUJ_20211208_0_L2A'],
 '2021-12-13': ['S2A_45RUJ_20211213_0_L2A']}

In [49]:
lst = [(key, val, shape_file) for key, val in pids.items()]

In [5]:
ig = Indice()

In [ ]:
from multiprocessing import Pool, cpu_count, Process
pool = Pool(cpu_count()-2)
with pool:
    pool.map(ig.indices_generator, lst)

In [50]:
def merge_clip_raster(raster_file_list, output_file=None, shp_file=None, bbox=None, out_width=None, out_height=None):
    ds_lst = list()
    for raster in raster_file_list:
        ds = gdal.Warp('', raster, format='vrt', dstNodata=0,
                       dstSRS="+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0",
                       cutlineDSName=shp_file, cropToCutline=True, width=out_width, height=out_height)
        ds_lst.append(ds)
    del ds
    if shp_file:
        ds = gdal.BuildVRT('', ds_lst, VRTNodata=0, srcNodata=0)
        gdal.Warp(output_file, ds, format='GTiff', dstNodata=0,
                  dstSRS="+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0",
                  cutlineDSName=shp_file, cropToCutline=True, width=out_width, height=out_height)
    else:
        if out_width:
            ds = gdal.BuildVRT('', ds_lst, VRTNodata=0, srcNodata=0)
            gdal.Warp(output_file, ds, format='GTiff', dstNodata=0,
                      dstSRS="+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0",
                      outputBounds=tuple(bbox), cropToCutline=True, width=out_width, height=out_height)
        else:
            ds = gdal.BuildVRT('', ds_lst, VRTNodata=0, srcNodata=0)
            gdal.Warp(output_file, ds, format='GTiff', dstNodata=0,
                      dstSRS="+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0",
                      outputBounds=tuple(bbox), cropToCutline=True)
    return output_file

In [ ]:
loc_fp = '/vsimem/ndvi.tif'
merge_clip_raster(['data/file/indices/2021-11-03/ndvi.tif'], output_file=loc_fp, shp_file=shape_file)

In [ ]:
os.makedirs('data/agrimask')

In [ ]:
from glob import glob
files = glob('data/file/indices/2021-11-03/ndvi.tif')

In [39]:
arr = raster_to_array('data/agrimask/vaishali1.tif', data_type='int64')
arr1 = raster_to_array('ndvi.tif', data_type='float')

In [27]:
ds = gdal.Open('vaishali.img')

In [ ]:
ds.RasterYSize

In [ ]:
arr.shape, arr1.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(arr)

In [ ]:
plt.imshow(arr1)

In [ ]:
arr1[arr==0] = 0

In [ ]:
plt.imshow(arr1)

In [33]:
arr[2912][2112]

40

In [40]:
arr = arr.astype('float')


In [43]:
arr[2912][2112]

nan

In [42]:
arr[arr==40] = np.nan

In [44]:
def upload_to_s3(mem_fp, s3_fp, bucket_name):
    f = gdal.VSIFOpenL(mem_fp, 'rb')
    gdal.VSIFSeekL(f, 0, 2)  # seek to end
    size = gdal.VSIFTellL(f)
    gdal.VSIFSeekL(f, 0, 0)  # seek to beginning
    data = gdal.VSIFReadL(1, size, f)
    gdal.VSIFCloseL(f)
    # Upload the raw data to s3
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)
    s3.put_object(Key=s3_fp, Bucket=bucket_name, Body=data, ContentLength=size)